In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch


In [2]:
# Step 2: Load the FINBERT model
model_name = "yiyanghkust/finbert-tone"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [4]:
import os

# Specify the current file name
current_file_name = "shadow_minutes 9_rajat.csv"

# Specify the new file name
new_file_name = "test.csv"

# Rename the file
os.rename(current_file_name, new_file_name)

In [5]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("test.csv")

# Print the column names
print("Column Names:")
print(df.columns)

Column Names:
Index(['Sentences', 'Dominant Sentiment', 'Reason'], dtype='object')


In [6]:
# Step 3: Load and preprocess your data
def preprocess_data(csv_file):
    df = pd.read_csv(csv_file)
    # Your preprocessing steps here
    return df

# Step 4: Perform sentiment analysis
def perform_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).detach().numpy()
    return probabilities

# Step 5: Evaluate the results

# Example usage
csv_file = "/content/test.csv"
data = preprocess_data(csv_file)
sentiments = []
positive_probs = []
negative_probs = []
neutral_probs = []
for text in data['Sentences']:
    probabilities = perform_sentiment_analysis(text)
    positive_prob = round(probabilities[0][1], 2)  # Round to two decimal places
    negative_prob = round(probabilities[0][0], 2)  # Round to two decimal places
    neutral_prob = round(probabilities[0][2], 2)

    if positive_prob > negative_prob and positive_prob > neutral_prob:
        sentiment = "positive"
    elif negative_prob > positive_prob and negative_prob > neutral_prob:
        sentiment = "negative"
    else:
        sentiment = "neutral"

    sentiments.append(sentiment)
    positive_probs.append(float(positive_prob))  # Convert numpy float32 to Python float
    negative_probs.append(float(negative_prob))  # Convert numpy float32 to Python float
    neutral_probs.append(float(neutral_prob))  # Convert numpy float32 to Python float

data['sentiment'] = sentiments
data['positive_prob'] = positive_probs
data['negative_prob'] = negative_probs
data['neutral_prob'] = neutral_probs

print(data)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                             Sentences Dominant Sentiment  \
0    There is an unprecedented shock to every econo...          Negative    
1    It practically immobilizes the workforce and t...          Negative    
2    As in the previous Financial Crisis, the Monet...            Neutral   
3    RBI must ensure that key Institutions (like Ba...            Neutral   
4    The last also requires close Monetary-Fiscal C...            Neutral   
..                                                 ...                ...   
158  While the current fiscal and monetary actions ...           Positive   
159  Going forward, more fiscal and monetary measur...            Neutral   
160  Right now, control and coordination to prevent...            Neutral   
161  RBI's initiatives are obviously in the right d...           Positive   
162  Only issue for discussion is if the measures a...            Neutral   

                                                Reason sentiment  \
0      

In [24]:
# Example usage
statement = input("Enter a statement: ")
sentiment, probabilities = perform_sentiment_analysis(statement)
print(f"Predicted Sentiment: {sentiment}")
print(f"Probability (Positive): {round(probabilities[0][1],2)}")
print(f"Probability (Negative): {round(probabilities[0][0],2)}")
print(f"Probability (Neutral): {round(probabilities[0][2],2)}")


Enter a statement: inflation is rising 
Predicted Sentiment: Negative
Probability (Positive): 0.07000000029802322
Probability (Negative): 0.6200000047683716
Probability (Neutral): 0.3100000023841858
